In [69]:
import numpy as np
import math 
import matplotlib.pyplot as plt
%matplotlib notebook

# Ларичева Мария БПМ-161
## Лабораторная работа 3. Вариант 12

### **Задача 3.1.**
<br>Дана система уравнений $Ax = b$ порядка n. Исследовать зависимость погрешности решения $x$ от погрешностей правой части системы $b$.
<br>
<br>Компоненты вектора $b$ во всех вариантах задаются формулой $b_i = N, ∀i = 1..n$, коэффициенты $c = c_{ij} = 0.1*N*i*j, ∀i,j = 1..n$. $N$ - номер варианта
<br>
<br>Матрица $A$: $A_{ij} = \frac{111}{c^4+13+3c}$


Зададим матрицу системы $A$ и вектор правой части $b$.

In [70]:
N = 4
A = np.empty([N, N])
b = np.ones(N)
b = b*12
for i in range(N):
    for j in range(N):
        c = 0.1*12*i*j
        A[i][j] = 111/(c**4+13+3*c)

print("матрица A:", A)
print("вектор b", b)

матрица A: [[8.53846154e+00 8.53846154e+00 8.53846154e+00 8.53846154e+00]
 [8.53846154e+00 5.94422072e+00 2.07952400e+00 5.78843731e-01]
 [8.53846154e+00 2.07952400e+00 1.98838639e-01 4.07790548e-02]
 [8.53846154e+00 5.78843731e-01 4.07790548e-02 8.13169561e-03]]
вектор b [12. 12. 12. 12.]


Найдем решение системы методом Гаусса через встроенную функцию *lsolve*

In [71]:
x = np.linalg.solve(A, b)
print(x)

[ 1.40540541 -0.         -0.          0.        ]


Найдем **число обусловленности** матрицы с помощью встроенной функции *condi*. Число обусловленности определяет, насколько погрешность входных данных может повлиять на решение системы

In [72]:
cnumb =  np.linalg.cond(A)
print("condition number for matrix A:", cnumb)

condition number for matrix A: 107.11394559222482


Найдем вектор относительных погрешностей $d$. 
<br>$d^i$ = $\frac{||x - x^i||}{||x||}$, где $x^i$ - решение системы $Ax^i$ = $b^i$, $b^i_j = b^i_j + eps$, если $j = i$. <br>$eps$ -- произвольная величина, возьмем $eps = 0.01$

In [73]:
eps = 0.01
def func(A, b): #функция, рассчитывающая вектор d
    d = np.ones(N)
    for i in range(N):
        b_i = np.ones(N)
        for j in range(N):
            if (j == i):
                b_i[j] = b[j]+eps
            else:
                b_i[j] = b[j]
        x_i = np.linalg.solve(A, b_i)
        d_i = np.linalg.norm(x-x_i, np.inf)/np.linalg.norm(x, np.inf)
        d[i] = d_i
    return d
d = func(A,b)

Найдем компоненту $b_m$ вектора $b$, которая оказывает наибольшее влияние на погрешность решения. Для этого прибавим некоторое значение $c$ (в данном случае, $c = 1$) к каждой из компонент вектора b и оценим норму вектора относительных погрешностей $d$. 

In [74]:
lst = []
for i in range(N):
    b_m = np.ones(N)*12
    b_m[i] += 1
    lst.append(np.linalg.norm(func(A, b_m)))

print("нормы вектора d",lst)
max_value = max(lst)
max_index = lst.index(max_value)
print("компонента вектора, весящая больше всего (при условии, что первая компонента имеет порядковый номер 0): ",max_index)

нормы вектора d [0.2381263380355069, 1.3459192633409638, 4.4233925795946485, 3.1508986908595324]
компонента вектора, весящая больше всего (при условии, что первая компонента имеет порядковый номер 0):  2


Оценим теоретически погрешность решения $x^m$ по формуле $\delta(x^m)$ $\leq$ $cond(A)\delta(b^m)$. Вычислим $x^m$ - решение системы $Ax = b^m$

In [75]:
b_m = np.ones(N)
for j in range(N):
    if (j == i):
        b_m[j] = b[j]+eps
    else:
        b_m[j] = b[j]
x_m = np.linalg.solve(A, b_m)
print("x = ", x_m)

x =  [ 1.40707276 -0.0086662   0.02214058 -0.01514173]


In [76]:
dlt_m = np.linalg.norm(b-b_m, np.inf)/np.linalg.norm(b, np.inf)

print("delta(x^m) <= ",dlt_m*cnumb) 
print("d_m = ", d[max_index])

delta(x^m) <=  0.08926162132685211
d_m =  0.02211652051451146


Получилось, что значение практической погрешности выше, чем погрешность решения, оценненая по формуле выше (теоретическая). СЛАУ является плохо обусловленной, на что также указывает число обусловленности 

### **Задача 3.2.** 
Для системы уравнений $Ax = b$ из задачи 3.1 исследовать зависимость погрешности решения системы от погрешностей коэффициентов матрицы $A$ (аналогично задаче 3.1).

Для нахождения элемента матрицы $A$, оказывающего максимальное влияние на погрешность решения системы, проведем аналогичные предыдущему пункту расчеты. Добавим к каждому из элементов некоторое фиксированное значение и вычислим погрешность $d$ 

In [77]:
lst = []
max_value = 0
for i in range(N):
    for j in range(N):
        A_m = np.copy(A)
        A_m[i][j]+=1
        elem = np.linalg.norm(func(A_m, b))
        lst.append(elem)
        if (elem >= max(lst)):
            max_value = elem
print("лист погрешностей d: ", lst)

лист погрешностей d:  [0.3341654983528615, 0.02754263691647458, 0.029578822931526645, 0.027113264756128564, 1.8883752414467825, 0.037234850877163314, 0.0144150856037444, 0.17033791341190382, 6.554588964707342, 0.014113641872560364, 0.013293369841166133, 0.00894988402485142, 3.7961852449994375, 0.17921939881924884, 0.009021327953001592, 0.058690992267004846]


Переформируем лист из погрешностей в матрицу размерности $4х4$ и найдем положение элемента $max$_$value$

In [78]:
eps = 0.01
matrix = np.array(lst).reshape(4,4)
print("matrix: ", matrix)
(ind1, ind2) = np.where(matrix == max_value) 
print("положение искомого элемента: ", ind1, ind2)

matrix:  [[0.3341655  0.02754264 0.02957882 0.02711326]
 [1.88837524 0.03723485 0.01441509 0.17033791]
 [6.55458896 0.01411364 0.01329337 0.00894988]
 [3.79618524 0.1792194  0.00902133 0.05869099]]
положение искомого элемента:  [2] [0]


Составим возмущенную матрицу $A$_$max$

In [79]:
A_max = np.copy(A)
for i in range(N):
    for j in range(N):
        if (i == ind1) and (j == ind2):
            A_max[i][j] += eps
A_max = np.array(A_max)
print("A* = ", A_max)

A* =  [[8.53846154e+00 8.53846154e+00 8.53846154e+00 8.53846154e+00]
 [8.53846154e+00 5.94422072e+00 2.07952400e+00 5.78843731e-01]
 [8.54846154e+00 2.07952400e+00 1.98838639e-01 4.07790548e-02]
 [8.53846154e+00 5.78843731e-01 4.07790548e-02 8.13169561e-03]]


Проведем оценку погрешности решения системы по формуле $\delta(x^m)$ $\leq$ $cond(A)\delta(A^*)$

In [80]:
dlt_A = np.linalg.norm(A-A_max, np.inf)/np.linalg.norm(A, np.inf)
print("delta(x^m) <= ",dlt_A*cnumb) 

delta(x^m) <=  0.03136219127700209


### Задача 3.5.3
Решить систему уравнений $Ax = b$ порядка $n$ из задачи $3.5$ методом Холецкого. Вычислить число обусловленности матрицы $A$

In [81]:
n = 30
m = 9
A3 = np.empty([n, n])
b3 = [i**2 - 100 for i in range(n)]
b3 = np.array(b3)
for i in range(n):
    for j in range(n):
        if (i == j):
            A3[i][j] = n + m**2 + j/m + i/n
        else:
            A3[i][j] = (i+j)/(m+n)

Решим систему методом Холецкого. Для симметричных положительно определенных матриц этот метод позволяет найти LU-разложение $A = LL^T$, где $L$ - треугольная матрица

In [82]:
L = np.linalg.cholesky(A3)
razl  = np.dot(L, L.T.conj())
#print("разложение матрицы А: ", razl)

Найдем решение системы $Ly = b$

In [83]:
y = np.linalg.solve(L,b3)
print("y = ",y)

y =  [-9.49157996 -9.38836529 -9.08921376 -8.59486765 -7.90638557 -7.02513453
 -5.9527797  -4.69127212 -3.24283442 -1.60994474  0.20468083  2.19810722
  4.3672004   6.70864834  9.21898248 11.8945997  14.73178441 17.72673054
 20.87556334 24.17436063 27.61917352 31.20604626 34.93103517 38.79022664
 42.77975382 46.8958123  51.13467438 55.49270219 59.96635936 64.55222151]


Найдем решение системы $L^Tx = y$

In [84]:
x3 = np.linalg.solve(L.T.conj(), y)
print("x = ", x3)

x =  [-1.15589719 -1.1546237  -1.13536451 -1.09816481 -1.04306961 -0.97012379
 -0.87937207 -0.77085901 -0.64462904 -0.50072644 -0.33919533 -0.16007969
  0.03657663  0.25072996  0.48233674  0.73135359  0.99773724  1.28144457
  1.58243263  1.90065857  2.23607973  2.58865354  2.9583376   3.34508965
  3.74886756  4.16962934  4.60733314  5.06193725  5.53340009  6.02168022]


Сравним его с решением системы $Ax = b$. Полученные решения совпадают с некоторой точностью

In [85]:
new_x = np.linalg.solve(A3, b3)
print(new_x)

[-1.15589719 -1.1546237  -1.13536451 -1.09816481 -1.04306961 -0.97012379
 -0.87937207 -0.77085901 -0.64462904 -0.50072644 -0.33919533 -0.16007969
  0.03657663  0.25072996  0.48233674  0.73135359  0.99773724  1.28144457
  1.58243263  1.90065857  2.23607973  2.58865354  2.9583376   3.34508965
  3.74886756  4.16962934  4.60733314  5.06193725  5.53340009  6.02168022]
